In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

all_colors = px.colors.qualitative.Plotly

In [2]:
layout_settings = {
    "width": 720, 
    "height": 480,
    "font": dict(size=16),
    "margin": dict(l=10, r=10, t=10, b=10),
}

def get_rgba(hex, opacity=0.3):
    return "rgba(%d, %d, %d, %.2f)" % (tuple(int(hex.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) + (opacity,))

In [3]:
df_trials = [
    [
        pd.read_csv("res/39-40.full/39.nh.1.1.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.2.1.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.3.1.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.4.1.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.5.1.csv", header=0),
    ], 
    [
        pd.read_csv("res/39-40.full/39.nh.1.2.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.2.2.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.3.2.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.4.2.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.5.2.csv", header=0),
    ],
    [
        pd.read_csv("res/39-40.full/39.nh.1.3.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.2.3.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.3.3.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.4.3.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.5.3.csv", header=0),        
    ], 
    [
        pd.read_csv("res/39-40.full/39.nh.1.4.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.2.4.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.3.4.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.4.4.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.5.4.csv", header=0),
    ],
    [
        pd.read_csv("res/39-40.full/39.nh.1.5.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.2.5.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.3.5.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.4.5.csv", header=0),
        pd.read_csv("res/39-40.full/39.nh.5.5.csv", header=0),
    ],
]

In [4]:
runs = []
for threads in [1, 2, 4, 8, 12, 16, 8704]:
    trials = []
    for trial, trial_group in enumerate(df_trials):
        rows = []
        for radius, radius_df in enumerate(trial_group):
            # get avgEvolveTime for this thread
            row = radius_df[['avgEvolveTime']].loc[radius_df['threads'] == threads].iloc[0]
            # include radius, which is equivalent to this df index, and avgEvolveTime
            rows.append([radius + 1, row[0]])
        # create a df that represents a trial for this number of threads, where x = 'radius'
        trial_df = pd.DataFrame(data=rows, columns=['radius', 'avgEvolveTime'])
        trials.append(trial_df)
    # append all trials for this number of threads
    runs.append((f"{threads} CPUs", trials, (threads,),))

runs[0] = ('1 CPU',) + runs[0][1:]
runs[-1] = ('GPU',) + runs[-1][1:]

cpu_df = pd.concat(runs[0][1])
cpu_df = cpu_df.groupby(cpu_df.index).mean() # use cpu_df mean only

runs_parallel = runs[1:]

In [5]:
def add_mean_std_trace(fig, df, name, idx, y_col, x_col="size"):
    df_mean = df.groupby(df.index).mean()
    df_std = df.groupby(df.index).std()

    color = all_colors[idx % len(all_colors)]
    fig.add_trace(go.Scatter(
        name=name,
        x=df_mean[x_col],
        y=df_mean[y_col],
        mode="lines+markers",
        line=dict(color=color),
    ))
    fig.add_traces(go.Scatter(
        name=name + " upper",
        x=df_mean[x_col],
        y=df_mean[y_col] + df_std[y_col],
        mode="lines",
        line=dict(width=0),
        showlegend=False
    ))
    fig.add_traces(go.Scatter(
        name=name + " lower",
        x=df_mean[x_col],
        y=df_mean[y_col] - df_std[y_col],
        line=dict(width=0),
        mode='lines',
        fillcolor=get_rgba(color),
        fill='tonexty',
        showlegend=False,
    ))

In [11]:
fig = go.Figure()
for idx, (name, dfs, _) in enumerate(runs):
    df = pd.concat(dfs)
    df["avgEvolveTime"] = df["avgEvolveTime"] / 1e6 # time is originally in ns

    add_mean_std_trace(fig, df, name, idx, "avgEvolveTime", x_col="radius")

fig.update_layout(xaxis_title='NH Radius', yaxis_title="Time (ms)", **layout_settings) # title="Average grid evolution time", 
fig.update_xaxes(tickformat='d')
fig.update_yaxes(type="log")
fig.show()
fig.write_image("images/nh_hl_evo_avg.eps")

In [9]:
fig = go.Figure()

for idx, (name, dfs, _) in enumerate(runs_parallel):
    for i, df in enumerate(dfs):
        dfs[i]["speedup"] = cpu_df["avgEvolveTime"] / df["avgEvolveTime"]

    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx + 1, "speedup", x_col="radius")

fig.update_layout(xaxis_title='NH Radius', yaxis_title="Speedup", **layout_settings) # title="Speedup vs 1 CPU", 
fig.update_xaxes(tickformat='d')
fig.update_yaxes(type="log")
fig.show()
fig.write_image("images/nh_hl_evo_speedup.eps")

In [10]:
fig = go.Figure()

for idx, (name, dfs, info) in enumerate(runs_parallel):
    for i, df in enumerate(dfs):
        dfs[i]["efficiency"] = 100 * (cpu_df["avgEvolveTime"] / df["avgEvolveTime"]) / info[0]
    
    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx + 1, "efficiency", x_col="radius")

fig.update_layout(xaxis_title='NH Radius', yaxis_title="Efficiency %", **layout_settings) # title="Efficiency", 
fig.update_xaxes(tickformat='d')
fig.show()
fig.write_image("images/nh_hl_evo_efficiency.eps")